In [27]:
%env KERAS_BACKEND = tensorflow

env: KERAS_BACKEND=tensorflow


In [28]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np


from keras.datasets import imdb

from keras.models import Sequential
from keras.layers import Dense, Embedding,Dropout
from keras.layers import LSTM
import keras.callbacks

In [29]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

In [30]:
print('訓練總筆數',len(x_train))
print('測試總筆數',len(x_test))

訓練總筆數 25000
測試總筆數 25000


In [31]:
from keras.preprocessing import sequence

In [32]:
#pad_sequences若資料太長則截斷，太短則補0
x_train = sequence.pad_sequences(x_train, maxlen=150)
x_test = sequence.pad_sequences(x_test, maxlen=150)

In [33]:
x_train.shape

(25000, 150)

In [34]:
N = 10 # 文字要壓到 N 維
K = 50 # LSTM 有 K 個神經元

In [44]:
model = Sequential()

model.add(Embedding(10000, N))

model.add(LSTM(K))

model.add(Dense(1, activation='sigmoid'))

In [45]:
model.compile(loss='binary_crossentropy',
              optimizer='RMSProp',metrics=['accuracy'])

In [46]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, None, 10)          100000    
_________________________________________________________________
lstm_13 (LSTM)               (None, 50)                12200     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 51        
Total params: 112,251
Trainable params: 112,251
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.fit(x_train, y_train,batch_size=32,epochs=5)

Epoch 1/5
25000/25000 [==============================] - 40s 2ms/step - loss: 0.4396 - acc: 0.7938
Epoch 2/5
25000/25000 [==============================] - 40s 2ms/step - loss: 0.2835 - acc: 0.8845
Epoch 3/5
25000/25000 [==============================] - 39s 2ms/step - loss: 0.2487 - acc: 0.9015
Epoch 4/5
25000/25000 [==============================] - 38s 2ms/step - loss: 0.2299 - acc: 0.9110
Epoch 5/5
25000/25000 [==============================] - 40s 2ms/step - loss: 0.2147 - acc: 0.9196


In [48]:
score = model.evaluate(x_test, y_test)

25000/25000 [==============================] - 8s 314us/step


In [49]:
print(f'測試資料的 loss = {score[0]}')
print(f'測試資料正確率 = {score[1]*100}%')

測試資料的 loss = 0.4169336189270019
測試資料正確率 = 81.452%


#### 這邊只修改了文字壓縮的維度、LSTM神經元個數以及優化器採用keras推薦適合RNN的RMSProp

#### 測試資料表現不如訓練資料，表示有overfitting的狀況產生，以下加入dropout層和earlystop方法來避免overfitting

### 先單純增加dropout層

In [50]:
N = 10 # 文字要壓到 N 維
K = 50 # LSTM 有 K 個神經元

model1 = Sequential()
model1.add(Embedding(10000, N))
model1.add(Dropout(0.2))#當中的rate代表丟棄的數據比例，以0~1來表示
model1.add(LSTM(K))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid'))

In [51]:
model1.compile(loss='binary_crossentropy',
              optimizer='adam',metrics=['accuracy'])

In [52]:
model1.fit(x_train, y_train,batch_size=32,epochs=5,validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 50s 2ms/step - loss: 0.4425 - acc: 0.7786 - val_loss: 0.3231 - val_acc: 0.8640
Epoch 2/5
25000/25000 [==============================] - 47s 2ms/step - loss: 0.2639 - acc: 0.8962 - val_loss: 0.3221 - val_acc: 0.8661
Epoch 3/5
25000/25000 [==============================] - 50s 2ms/step - loss: 0.2162 - acc: 0.9186 - val_loss: 0.3486 - val_acc: 0.8624
Epoch 4/5
25000/25000 [==============================] - 51s 2ms/step - loss: 0.1884 - acc: 0.9306 - val_loss: 0.3673 - val_acc: 0.8565
Epoch 5/5
25000/25000 [==============================] - 49s 2ms/step - loss: 0.1684 - acc: 0.9384 - val_loss: 0.4341 - val_acc: 0.8430


In [53]:
score = model1.evaluate(x_test, y_test)
print(f'測試資料的 loss = {score[0]}')
print(f'測試資料正確率 = {score[1]*100}%')

25000/25000 [==============================] - 7s 288us/step
測試資料的 loss = 0.43407998836040496
測試資料正確率 = 84.304%


### 沒有改善過適的問題，試試看earlystop吧

In [54]:
N = 10 # 文字要壓到 N 維
K = 50 # LSTM 有 K 個神經元

model2 = Sequential()
model2.add(Embedding(10000, N))
model2.add(Dropout(0.2))#當中的rate代表丟棄的數據比例，以0~1來表示
model2.add(LSTM(K))
model2.add(Dropout(0.2))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy',
              optimizer='adam',metrics=['accuracy'])
EarlyStop = keras.callbacks.EarlyStopping(monitor='val_loss',mode='auto')
model2.fit(x_train,y_train,batch_size =32,epochs = 10,validation_data = (x_test, y_test),callbacks=[EarlyStop])

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 51s 2ms/step - loss: 0.4445 - acc: 0.7804 - val_loss: 0.3232 - val_acc: 0.8617
Epoch 2/10
25000/25000 [==============================] - 49s 2ms/step - loss: 0.2707 - acc: 0.8940 - val_loss: 0.3132 - val_acc: 0.8636
Epoch 3/10
25000/25000 [==============================] - 48s 2ms/step - loss: 0.2215 - acc: 0.9154 - val_loss: 0.3230 - val_acc: 0.8635


In [55]:
score = model2.evaluate(x_test, y_test)
print(f'測試資料的 loss = {score[0]}')
print(f'測試資料正確率 = {score[1]*100}%')

25000/25000 [==============================] - 7s 290us/step
測試資料的 loss = 0.32298372346401216
測試資料正確率 = 86.35199999999999%


In [56]:
N = 10 # 文字要壓到 N 維
K = 50 # LSTM 有 K 個神經元

model3 = Sequential()
model3.add(Embedding(10000, N))
model3.add(Dropout(0.3))#當中的rate代表丟棄的數據比例，以0~1來表示
model3.add(LSTM(K))
model3.add(Dropout(0.3))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy',optimizer='RMSProp',metrics=['accuracy'])
earlystop = keras.callbacks.EarlyStopping(monitor='val_loss',mode='auto')
model3.fit(x_train,y_train,batch_size =32,epochs = 5,validation_data = (x_test, y_test),callbacks=[earlystop])

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 52s 2ms/step - loss: 0.4568 - acc: 0.7857 - val_loss: 0.3289 - val_acc: 0.8625
Epoch 2/5
25000/25000 [==============================] - 49s 2ms/step - loss: 0.2986 - acc: 0.8774 - val_loss: 0.3207 - val_acc: 0.8688
Epoch 3/5
25000/25000 [==============================] - 49s 2ms/step - loss: 0.2672 - acc: 0.8957 - val_loss: 0.2980 - val_acc: 0.8729
Epoch 4/5
25000/25000 [==============================] - 49s 2ms/step - loss: 0.2485 - acc: 0.9042 - val_loss: 0.3285 - val_acc: 0.8639


In [57]:
score = model3.evaluate(x_test, y_test)
print(f'測試資料的 loss = {score[0]}')
print(f'測試資料正確率 = {score[1]*100}%')

25000/25000 [==============================] - 8s 307us/step
測試資料的 loss = 0.32846082658290865
測試資料正確率 = 86.388%


### 這邊結合一開始使用的RMSProp，dropout層以及earlystop的方法，雖然防止了overfitting的發生，但讓測試資料的正確率降低了